In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

script_dir = Path('__file__').resolve().parent


In [17]:
data_path = script_dir.joinpath('data')

In [20]:
df_holidays = pd.read_csv(data_path.joinpath('holidays_events.csv'))
df_oil = pd.read_csv(data_path.joinpath('oil.csv'))
df_stoes = pd.read_csv(data_path.joinpath('stores.csv'))
df_tansactions = pd.read_csv(data_path.joinpath('transactions.csv'))
df_test = pd.read_csv(data_path.joinpath('test.csv'))
df_tain = pd.read_csv(data_path.joinpath('train.csv'))